In [1]:
# tiff > portable movie viewer 

# Cyna Shirazinejad
# 6/19/20

# capabilities needed:
# - auto-segmentation (once we establish: organelles we'll be working with, expression levels, cell types, etc.)
# - auto-tracking of puncta (CME), blobs (organelles) that is then overlaid on images
# - text annotation: scale bar, channel names (auto-extracted from image title), markers, imaging settings (exposure, frame rate, intensity), conditions, timestamp (minute and seconds)


# pulse chase/stimulation: digital toggle indicating where 
# - photobleaching correction (probably exponential fit to start with)
# - extension/generalization to other data formats (TIFFS), Hd5



In [2]:
# not necessary unless running vm
# !source ~/.bash_profile


In [8]:

# import all necessary Python libraries

# configure the Jupyter Notebook
%load_ext autoreload
%autoreload 2
%matplotlib inline
%gui qt

import pickle
import napari 
import imageio
import os
import sys
# import z5py

import numpy as np
from os import listdir
from os.path import isfile, join
from ipywidgets import interact, interactive, fixed, interact_manual # for interactive widgets
import dask.array as da
import dask
import tifffile
import subprocess
import scipy as sp
import scipy.optimize
sys.path.append(os.path.abspath(os.getcwd() + '/visualization_python_scripts_cyna'))
from visualization_nd2 import return_widgets_for_display, make_movie
from dask_to_napari import (convert_tiffs_to_dask_arrays, 
                            view_one_channel, 
                            view_one_time_one_channel,
                            view_specified_channels_one_time,
                            populate_grid)
from visualization_utilities import add_3d_axes
from skimage.io import imread
from dask import delayed
import tifffile
import dask.array as da
from glob import glob
from PIL import Image
from pims import ImageSequence
# import dask_image
import imageio

!jupyter nbextension enable --py widgetsnbextension


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [4]:
# # trying dask_image from: https://blog.dask.org/2019/06/20/load-image-data
# import dask_image
# filename_dask_image_test = '/Users/cynashirazinejad/Desktop/llsm_test_data/CPPdecon/exp08_CamB_ch0_CAM1_stack0000_560nm_0000000msec_0002628334msecAbs_000x_000y_000z_0000t_decon.tif'
# sample_dask_image = dask_image.imread.imread(filename_dask_image_test)

# # RESULTS IN AN ERROR?
# # AttributeError: module 'dask_image' has no attribute 'imread'
# # unexpected because dask_image. + tab shows imread as a module, and imread as a function in the imread module

In [5]:
# trying the imageio library
import imageio
filename_imageio_test = '/Volumes/GoogleDrive/My Drive/Drubin Lab/Imaging/clusterfs_fiona_Data_20200715_p35_p4_WTC_iPS_AP2rfp_Arp3ceGFP_nsp3_Exp08_3color_200mW_z0p5_DS_CPPdecon/exp08_CamA_ch0_CAM1_stack0000_560nm_0000000msec_0002628334msecAbs_000x_000y_000z_0000t_decon.tif'
sample_imageio = imageio.imread(filename_imageio_test)
print(sample_imageio.shape) # yields an incorrect image size (does not capture Z); yields (1600,1099)

(1600, 1099)


In [4]:
# trying the tifffile library
%time
filename_tifffile_test = '/Volumes/GoogleDrive/My Drive/Drubin Lab/Imaging/clusterfs_fiona_Data_20200715_p35_p4_WTC_iPS_AP2rfp_Arp3ceGFP_nsp3_Exp08_3color_200mW_z0p5_DS_CPPdecon/exp08_CamA_ch0_CAM1_stack0000_560nm_0000000msec_0002628334msecAbs_000x_000y_000z_0000t_decon.tif'
import tifffile
sample_tifffile = tifffile.imread(filename_tifffile_test)
print(sample_tifffile.shape) # yields the expected image shape ((151,1600,1099))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 2.86 µs
(151, 1600, 1099)


In [5]:
%time
filename_scikit_image_test = '/Volumes/GoogleDrive/My Drive/Drubin Lab/Imaging/clusterfs_fiona_Data_20200715_p35_p4_WTC_iPS_AP2rfp_Arp3ceGFP_nsp3_Exp08_3color_200mW_z0p5_DS_CPPdecon/exp08_CamA_ch0_CAM1_stack0000_560nm_0000000msec_0002628334msecAbs_000x_000y_000z_0000t_decon.tif'
import skimage
sample_skimage = skimage.io.imread(filename_scikit_image_test)
print(sample_skimage.shape)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs
(151, 1600, 1099)


In [10]:
# will use tifffile as the image reading library from now onscipy

In [11]:
# import dask
# import dask.array as da

# lazy_arrays = [dask.delayed(imageio.imread)(fn) for fn in filenames]
# lazy_arrays = [da.from_delayed(x, shape=sample.shape, dtype=sample.dtype)
#                for x in lazy_arrays]

# lazy_array = dask.delayed(tifffile.imread)(filename)
# lazy_array = da.from_delayed(lazy_array, shape = sample.shape, dtype = sample.dtype)

# lazy_array.to_zarr('test.zarr')

# imgs = da.from_zarr('test.zarr',)




In [3]:
path_to_decon='/Volumes/GoogleDrive/My Drive/Drubin Lab/Imaging/clusterfs_fiona_Data_20200715_p35_p4_WTC_iPS_AP2rfp_Arp3ceGFP_nsp3_Exp08_3color_200mW_z0p5_DS_CPPdecon/'

In [4]:
# a list of lists. each internal list has filesnames for each channel
channels_dask = convert_tiffs_to_dask_arrays(path_to_decon,
                                             channel_identifiers=None,
                                             is_MIP=False,
                                             verbose=True)

Index of camera designation is 1
Index of channel designation is 2
Index of stack designation is 4
The stacks present in this file are: ['0000', '0001', '0002', '0003', '0004']
The channels will be returned in the following order linked to the detected identifiers: [[0, ('CamB', 'ch1')], [1, ('CamB', 'ch0')], [2, ('CamA', 'ch0')]]
Adding channel 0 to an output array
Adding stack 0 within channel 0 to an output array
Adding stack 1 within channel 0 to an output array
Adding stack 2 within channel 0 to an output array
Adding stack 3 within channel 0 to an output array
Adding stack 4 within channel 0 to an output array
Adding channel 1 to an output array
Adding stack 0 within channel 1 to an output array
Adding stack 1 within channel 1 to an output array
Adding stack 2 within channel 1 to an output array
Adding stack 3 within channel 1 to an output array
Adding stack 4 within channel 1 to an output array
Adding channel 2 to an output array
Adding stack 0 within channel 2 to an output arra

In [19]:
channels_dask

,Array,Chunk
Bytes,15.93 GB,101.40 MB
Shape,"(3, 5, 151, 1600, 1099)","(3, 5, 130, 100, 130)"
Count,289 Tasks,288 Chunks
Type,float32,numpy.ndarray


In [20]:
channels_dask.to_zarr("zarr_data")

In [9]:
channels_from_zarr=da.from_zarr("zarr_data")

In [7]:
channels_from_zarr

,Array,Chunk
Bytes,15.93 GB,101.40 MB
Shape,"(3, 5, 151, 1600, 1099)","(3, 5, 130, 100, 130)"
Count,289 Tasks,288 Chunks
Type,float32,numpy.ndarray


In [18]:
populate_grid(napari.Viewer(ndisplay=3), 1,1, [(channels_dask[0],[(0,0)],'magenta','ch0')])


In [27]:
populate_grid(napari.Viewer(ndisplay=3), 4 , 4, [(channels_dask[0],[(0,0),(3,3)],'magenta','ch0'),(channels_dask[1],[(1,1),(3,3)],'cyan','ch1'),(channels_dask[2],[(2,2),(3,3)],'yellow','ch2')])



In [ ]:
with napari.gui_qt():
    # add the image
    viewer=napari.Viewer(ndisplay=3)
    viewer.add_image(channels_from_zarr[0],colormap='viridis')
    shape_of_file = channels_from_zarr[0].shape
    text_properties = {
    'text': '{class}: {likelihood:0.1f}%',
    'anchor': 'upper_left',
    'translation': [-5, 0],
    'size': 8,
    'color': 'green',
    }

    # add polygons
    shapes_layer = viewer.add_shapes(
        polygons,
        properties=properties,
        shape_type='polygon',
        edge_width=3,
        edge_color='class',
        edge_color_cycle=edge_color_cycle,
        face_color='transparent',
        text=text_properties,
        name='shapes',
    )
    add_3d_axes(viewer, 
                shape_of_file,
                extension=[0,0,-600], 
                marker_width=3,
                offset=[0, 0, 0], 
                points_on_axes=(3,10,10))